### **Importing required libraries**

In [2]:
# import thr required libraries

import numpy as np
import pandas as pd
import os
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score




### **Declare the directory**

In [3]:
# check the current working directory
current_directory = os.getcwd()
print ("Current working directory : " , current_directory)

# define the directory of data set
data_direcory = "C:/Users/Avishka Sandeepa/OneDrive - University of Moratuwa/GitHub/Traffic-Signs-Recognition/main"
# chane the current directory to the data directory
os.chdir(data_direcory)

# chek the working directory again after changing
current_directory = os.getcwd()
print ("Current working directory : " , current_directory)

# define the each folder directories for training and testing data
train_set = 'Train'
test_set = 'Test'

num_classes = len(os.listdir(train_set)) # returns the no of classes inside the training folder(0 to 42)
num_classes

Current working directory :  c:\Users\Avishka Sandeepa\OneDrive - University of Moratuwa\GitHub\Traffic-Signs-Recognition\main\codes
Current working directory :  C:\Users\Avishka Sandeepa\OneDrive - University of Moratuwa\GitHub\Traffic-Signs-Recognition\main


43

### **Preprocessing the Images**

In [16]:
# Acess the each and every image

img_data =[]
img_labels = []
 
for index in range(num_classes):   # address all classes  
    path = os.path.join(current_directory,'Train',str(index)) # create the path for each image    
    image_names = os.listdir(path) # list containing names of all images for a running class at that point
    
    
    for image in image_names:
        img = cv.imread(path + '/' + image, cv.IMREAD_COLOR)
        img = cv.resize(img, (32,32)) # resizing all images to one scale
        # convert image data into numy array
        img = np.array(img) 

        img_data.append(img)
        img_labels.append(index)






In [17]:
# lets convert above created list into numpy arrays that helps to model to training

img_data = np.array(img_data)
img_labels = np.array(img_labels)

print(img_data.shape, img_labels.shape)

(39209, 32, 32, 3) (39209,)


In [19]:
# Split into training and validation data

train_x, val_x, train_y, val_y = train_test_split(img_data, img_labels, test_size=0.33, random_state=42, shuffle=True)
train_x = train_x/255 
val_x = val_x/255

print("train_x.shape", train_x.shape)
print("val_x.shape", val_x.shape)
print("train_y.shape", train_y.shape)
print("val_y.shape", val_y.shape)

train_x.shape (26270, 32, 32, 3)
val_x.shape (12939, 32, 32, 3)
train_y.shape (26270,)
val_y.shape (12939,)
